In [1]:
import torch
from transformers import pipeline
import gradio as gr
from langchain.llms import HuggingFaceHub
from transformers import pipeline
from langchain.prompts import PromptTemplate

from langchain.chains import LLMChain
from langchain_community.llms import HuggingFacePipeline

hf_pipeline = pipeline("summarization", model="facebook/bart-large-cnn")

# 2. Wrap it for LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)

temp = """
<s><<SYS>>
List the key points with details from the context: 
[INST] The context : {context} [/INST] 
<</SYS>>
"""
pt = PromptTemplate(
    input_variables=["context"],
    template= temp)
prompt_to_LLAMA2 = LLMChain(llm=llm, prompt=pt)

def transcript_audio(audio_file):
    # Initialize the speech recognition pipeline
    pipe = pipeline(
        "automatic-speech-recognition",
        model="openai/whisper-tiny.en",
        chunk_length_s=30,
        )
    
    # Transcribe the audio file and return the result
    transcript_txt = pipe(audio_file, batch_size=8)["text"]
    # run the chain to merge transcript text with the template and send it to the LLM
    result = prompt_to_LLAMA2.run(transcript_txt) 
    return result
# Set up Gradio interface
audio_input = gr.Audio(sources="upload", type="filepath")  # Audio input
output_text = gr.Textbox()  # Text output
# Create the Gradio interface with the function, inputs, and outputs
iface = gr.Interface(fn=transcript_audio, 
                     inputs=audio_input, outputs=output_text, 
                     title="Voice Summarization",
                     description="Upload the audio file")
# Launch the Gradio app
iface.launch(server_name="0.0.0.0", server_port=7860)

/Users/masoumehvojood/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/masoumehvojood/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['model.shared.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/05/m0z3j2y1647d9pk4d3932ks80000gn/T/ipykernel_57898/4041424415.py:14: LangChainDeprecationWarning: The class `Huggin

Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


/Users/masoumehvojood/Library/Python/3.9/lib/python/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
